<a href="https://colab.research.google.com/github/LLMUSER/on-device-tests/blob/main/Deploy_Streamlit_on_colab_QA_Using_LLAMA_2_13B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install langchain streamlit

In [45]:
!pip install -r requirements.txt

  Using cached aiofiles-23.2.1-py3-none-any.whl (15 kB)
  Using cached altair-5.0.1-py3-none-any.whl (471 kB)
  Using cached auto_gptq-0.2.0.tar.gz (47 kB)
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/b1/f9/97153ae5cf926f96fd37e61424a1bb58e0c9991cc220b2e17390fb8bde97/auto_gptq-0.2.0.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto-gptq==0.2.0 from https://files.pythonhosted.org/packages/b1/f9/97153ae5cf926f96fd37e61424a1bb58e0c9991cc220b2e17390fb8bde97/auto_gptq-0.2.0.tar.gz (from -r requirements.txt (line 10)) has inconsistent version: expected '0.2.0', but metadata has '0.2.0+cu1180'
ERROR: Could not find a version that satisfies the requirement auto-gptq==0.2.0 (from versions: 0.0.4, 0.0.5, 0.1.0, 0.2.0, 0.2.1, 0.2.2, 0.3.0, 0.3.1, 0.3.2, 0.4.2)
ERROR: No matching distribution found for auto-gptq==0.2.0


In [46]:
# !pip install streamlit
!pip install auto_gptq
!pip install transformers PyPDF2 docx2txt
!pip install langchain

In [52]:

DEFAULT_SYSTEM_PROMPT = """
    You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    """.strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

    #Here initializing Llama 2 13 B parameter model
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# model = AutoGPTQForCausalLM.from_quantized(
#     model_name_or_path,
#     revision="gptq-4bit-128g-actorder_True",
#     model_basename=model_basename,
#     use_safetensors=True,
#     trust_remote_code=True,
#     inject_fused_attention=False,
#     device=DEVICE,
#     quantize_config=None,
#     )

if "model" not in locals():
    model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        revision="gptq-4bit-128g-actorder_True",
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        inject_fused_attention=False,
        device=DEVICE,
        quantize_config=None,
    )





streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
    )

llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
    {context}

    Question: {question}
    """,
    system_prompt=SYSTEM_PROMPT,
    )

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    #Initializing the QA chain where we are passing our llm object our chroma db initializer vector db object and our prompts.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    )

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead',

NameError: ignored

In [55]:
%%writefile app.py
import streamlit as st
# from dotenv import load_dotenv
from htmlTemplates import css, bot_template, user_template
from langchain.document_loaders import DirectoryLoader,PyPDFLoader,UnstructuredMarkdownLoader,CSVLoader,UnstructuredHTMLLoader,UnstructuredExcelLoader,TextLoader,JSONLoader,Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, TextStreamer, pipeline
import csv
import openpyxl
from PyPDF2 import PdfReader
from langchain.document_loaders import OnlinePDFLoader
import docx2txt as dt
import pandas as pd
import tempfile



DEFAULT_SYSTEM_PROMPT = """
    You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    """.strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()


#preserve chat history and create response
def classify_and_process_file_and_create_vector_db(files):
    loaders =[]
    documents=[]
    for file in files:
        print(type(file))

        file_type = file.name.split('.')[-1]
        print(file)
        print(file_type)
        if file_type=="txt":
            temp_file = tempfile.NamedTemporaryFile(delete=False)

            temp_file.write(file.read())
            temp_file_path = temp_file.name
            # temp_file_path = os.path.join(temp_dir,temp_file.name)

            text_loader = TextLoader(file_path=temp_file_path).load()
            st.text("TXT file has been loaded into the text loader")
            print(text_loader)

            # loaders.append(text_loader)
            documents.extend(text_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
        if file_type == "pdf":
            temp_file = tempfile.NamedTemporaryFile(delete=False)
            temp_file.write(file.read())
            temp_file_path = temp_file.name
            # temp_file_path = os.path.join(temp_dir,temp_file.name)
            pdf_loader = PyPDFLoader(file_path=temp_file_path).load()
            # pdf_loader=DirectoryLoader(temp_dir, glob="**/*.pdf",loader_cls=PyPDFLoader)
            # data=pdf_loader.load()
            print(pdf_loader)
            st.text("Pdf has been successully loaded into the PDF_LOADER")
            # text = extract_text_from_pdf(temp_file_path)
            # print(text)
            st.write("Extracted PDF Text:")
            # st.write(text)
            # loaders.append(pdf_loader)
            documents.extend(pdf_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
        elif file_type == "docx":
            temp_file = tempfile.NamedTemporaryFile(delete=False)
            temp_file.write(file.read())
            temp_file_path = temp_file.name
            # temp_file_path = os.path.join(temp_dir,temp_file.name)
            docx_loader =  Docx2txtLoader(file_path=temp_file_path).load()
            # docx_loader=DirectoryLoader(temp_dir, glob="**/*.docx",loader_cls=Docx2txtLoader)
            st.text("DOCX has been successully loaded into the DOCX_LOADER")
            print(docx_loader)
            # text = extract_text_from_docx(temp_file_path)
            # print(text)

            st.write("Extracted DOCX Text:")
            # st.write(text)
            # loaders.append(docx_loader)
            documents.extend(docx_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
        elif file_type == "csv":
            temp_file = tempfile.NamedTemporaryFile(delete=False)
            temp_file.write(file.read())
            temp_file_path = temp_file.name

            # temp_file_path = os.path.join(temp_dir,temp_file.name)
            csv_loader = CSVLoader(file_path=temp_file_path,encoding='unicode_escape').load()
            # df = pd.read_csv(temp_file_path)
            # temp_file_path=df.to_excel(temp_file.name)
            # csv_loader = DataFrameLoader(temp_file_path).load()

            # csv_loader=DirectoryLoader(temp_dir, glob="**/*.csv",loader_cls=CSVLoader)
            st.text("CSV has been successully loaded into the CSV_LOADER")
            # time.sleep(0.5)s
            print(csv_loader)
            # dataframe = pd.read_csv(temp_file_path)
            # print(dataframe)
            # st.write("CSV Data:")
            # st.write(dataframe)
            # loaders.append(csv_loader)
            documents.extend(csv_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
        elif file_type == "xlsx":
            temp_file = tempfile.NamedTemporaryFile(delete=False)
            temp_file.write(file.read())
            temp_file_path = temp_file.name
            # temp_file_path = os.path.join(temp_dir,temp_file.name)
            excel_loader = UnstructuredExcelLoader(file_path=temp_file_path).load()
            # excel_loader=DirectoryLoader(temp_dir, glob="**/*.xlsx",loader_cls=UnstructuredExcelLoader)
            # st.text("Excel has been successully loaded into the DOCX_LOADER")
            print(excel_loader)
            print("Loaded the excel file in excel_loader")
            dataframe = pd.read_excel(temp_file_path, engine='openpyxl')
            print(dataframe)
            st.write("Excel Data:")
            st.write(dataframe)
            # loaders.append(excel_loader)
            documents.extend(excel_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
        elif file_type == "html":
            temp_file = tempfile.NamedTemporaryFile(delete=False)
            temp_file.write(file.read())
            temp_file_path = temp_file.name
            # temp_file_path = os.path.join(temp_dir,temp_file.name)
            html_loader = UnstructuredHTMLLoader(file_path=temp_file_path).load()
            # html_loader=DirectoryLoader(temp_dir, glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
            st.text("HTML has been successully loaded into the html_LOADER")
            print(html_loader)
            print("Loaded the html file in html_loader")
            # text = extract_text_from_html(temp_file_path)
            # print(text)
            st.write("Extracted HTML Text:")
            # st.write(text)
            # loaders.append(html_loader)
            documents.extend(html_loader)
            # temp_file.close()
            # if temp_file_path:
            #     os.remove(temp_file_path)
    print("Now loading the documents.")
    # documents = []
    # print(documents)
    # for loader in loaders:
    #     print(loader)
    #     documents.extend(loader.load())
    #     time.sleep(1.5)
    # print(documents)

    print (f'You have {len(documents)} document(s) in your data')
    print (f'There are {len(documents[0].page_content)} characters in your document')

    text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=200) #chunk overlap seems to work better
    documents = text_splitter.split_documents(documents)
    print(len(documents))

    #Generating the embeddings
    embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-tas-b")
    #Upserting into the ChromaDB which is a on device vector db
    # global vectorstore
    vectorstore = Chroma.from_documents(documents, embeddings,persist_directory="db")
    # return vectorstore
    st.text("Vector Database created and Indexed successfully !!")
    return vectorstore

def index_initializing_upserting_chroma_db(loaders):
    # pdf_loader = DirectoryLoader(Path(input_text), glob="**/*.pdf",loader_cls=PyPDFLoader)
    # readme_loader = DirectoryLoader(Path(input_text), glob="**/*.md",loader_cls=UnstructuredMarkdownLoader)
    # txt_loader = DirectoryLoader(Path(input_text), glob="**/*.txt",loader_cls=TextLoader)
    # csv_loader = DirectoryLoader(Path(input_text), glob="**/*.csv",loader_cls=CSVLoader)
    # html_loader = DirectoryLoader(Path(input_text), glob="**/*.html",loader_cls=s)
    # xlsx_loader=DirectoryLoader(Path(input_text), glob="**/*.xlsx",loader_cls=UnstructuredExcelLoader)
    # json_loader = DirectoryLoader(Path(input_text),glob="**/*.json",loader_cls=JSONLoader)
    # docx_loader =  DirectoryLoader(Path(input_text),glob="**/*.docx",loader_cls=Docx2txtLoader)

    #take all the loader


    #lets create document
    documents = []
    for loader in loaders:
        documents.extend(loader.load())
    print (f'You have {len(documents)} document(s) in your data')
    print (f'There are {len(documents[0].page_content)} characters in your document')

    text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=40) #chunk overlap seems to work better
    documents = text_splitter.split_documents(documents)
    print(len(documents))

    #Generating the embeddings
    embeddings = HuggingFaceEmbeddings(model_name="msmarco-distilbert-base-tas-b")
    #Upserting into the ChromaDB which is a on device vector db
    # global vectorstore
    vectorstore = Chroma.from_documents(documents, embeddings,persist_directory="db")
    return vectorstore







def create_conversation_chain(vectorstore):
    DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

    #Here initializing Llama 2 13 B parameter model

    model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
    model_basename = "model"

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    if "model" not in locals():


      model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        revision="gptq-4bit-128g-actorder_True",
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        inject_fused_attention=False,
        device=DEVICE,
        quantize_config=None,
    )




    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


    text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.2,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
    )

    llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


    SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

    template = generate_prompt(
    """
    {context}

    Question: {question}
    """,
    system_prompt=SYSTEM_PROMPT,
    )

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    #Initializing the QA chain where we are passing our llm object our chroma db initializer vector db object and our prompts.
    qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    )

    return qa_chain

def process_user_input(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        template = user_template if i % 2 == 0 else bot_template
        st.write(template.replace("{{MSG}}", message.content), unsafe_allow_html=True)





def main():
    # load_dotenv()
    st.set_page_config(page_title="Chat with multiple PDFs", page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    st.header("Chat with multiple PDFs :books:")
    user_question = st.text_input("Ask a question about your documents:")
    if user_question:
        process_user_input(user_question)

    with st.sidebar:
        st.subheader("Your documents")
        docs = st.file_uploader("Upload your PDFs here and click on 'Process'", accept_multiple_files=True,type=["pdf","docx","csv","xlsx","html","json"])
        if st.button("Process"):
            with st.spinner("Processing"):
                # raw_text = extract_text_from_pdfs(docs)
                # all_loaders =classify_and_load_files_into_respective_loaders(docs)
                # chroma_vectorstore = index_initializing_upserting_chroma_db(all_loaders)
                chroma_vectorstore=classify_and_process_file_and_create_vector_db(docs)
                st.text("Sucessfully Executed this function index_initializing_upserting_chroma_db")
                st.session_state.conversation = create_conversation_chain(chroma_vectorstore)
                st.success("Sucessfully Initialized LLama 2 13 B !!! You Can Chat with your files now !!")

if __name__ == '__main__':
    main()



Overwriting app.py


In [56]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2023-08-29 12:28:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.28M  5.39MB/s    in 2.5s    

2023-08-29 12:29:01 (5.39 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13921656/13921656]



In [57]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [58]:
!pip install temp docx2txt pip

In [59]:
!pip install pypdf

In [60]:
!pip install sentence_transformers chromadb

In [61]:
get_ipython().system_raw('./ngrok http 8501 &')

In [62]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://900e-35-247-130-51.ngrok-free.app


In [63]:
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.247.130.51:8501

2023-08-29 12:30:26.531413: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<class 'streamlit.runtime.uploaded_file_manager.UploadedFile'>
UploadedFile(file_id='8a4a42df-ff3c-461c-8b09-962f1358a14f', name='India_new.pdf', type='application/pdf', size=261815, _file_urls=file_id: "8a4a42df-ff3c-461c-8b09-962f1358a14f"
upload_url: "/_stcore/upload_file/74772406-108f-48ad-9f13-25dc6567c0e5/8a4a42df-ff3c-461c-8b09-962f1358a14f"
delete_url: "/_stcore/upload_file/74772406-108f-48ad-9f13-25dc6567c0e5/8a4a42df-ff3c-461c-8b09-962f1358a14f"
)
pdf
[Document(page_content="India, officially the Republic of India (ISO: Bhārat Gaṇarājya),[21] is a country in South Asia. It is the seventh-largest country by area; the most populous country as of June 2023;[22][23] and from the time of its independence in 1947, the wor